In [ ]:
import threading
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from openpyxl import load_workbook
from init_driver import init_driver
from count_dropdown_itens import count_dropdown_itens
from list_helper import list_pop, list_receiver
from get_daily_data import get_daily_data

wb = load_workbook('../BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

print("Coleta de CNPJs finalizada")

dados_diarios_list = []

def get_inside(cnpj, index, item_to_finish):
    try:
        print(f"Thread {index+1} iniciada para o CNPJ {cnpj}")
        driver = init_driver(index)
        print(f"Driver da Thread {index+1} iniciado com sucesso")

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
        )

        search_input.clear()
        
        search_input.send_keys(cnpj)
        search_input.send_keys(Keys.ENTER)
        print(f"Thread {index+1} pesquisou o CNPJ {cnpj}")

        try:
            find_name_to_store = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
            )
        except TimeoutException:
            find_name_to_store = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
            )

        stored_name = find_name_to_store.text
        find_name_to_store.click()

        driver.switch_to.default_content()

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        dados_diarios = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
        )
        dados_diarios.click()
        print(f"Thread {index+1} clicou em dados diários")

        driver.switch_to.default_content()

        #pop-up com mensagem irrelevante aparece e precisa ser fechado.
        try:
            alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
            alert.accept()
        except:
            pass

        dados_diarios_list.append(get_daily_data(index, driver, cnpj, stored_name, item_to_finish))

        
    except TimeoutException:
        print(f"Erro na Thread {index+1}")
        driver.quit()
        print(f"Thread {index+1} reiniciando...")
        get_inside(cnpj, index, item_to_finish)
    driver.quit()

list_cnpj_teste = ["58.878.941/0001-02", "36.248.874/0001-00", "51.017.442/0001-81", "57.270.020/0001-08"]

threads = []

# Criando threads em blocos de 8
for cnpj in list_cnpj_teste:
    print("Coleta tamanho do dropdown iniciada")
    tamanho = count_dropdown_itens(cnpj)
    print("Tamanho do dropdown coletado com sucesso")
    list_to_finish = list(range(tamanho))
    list_receiver(list_to_finish)
    for index in range(6):
        try:
            var = list_pop()
            thread = threading.Thread(target=get_inside, args=(cnpj, index, var))
            print(list_to_finish)
            threads.append(thread)
            thread.start()
        except IndexError:
            pass

# Aguardando todas as threads terminarem
for thread in threads:
    thread.join()

print("Todas as Threads foram finalizadas")

Driver da Thread 3 iniciado com sucesso
Driver da Thread 5 iniciado com sucesso
Erro no carregamento da Thread 1, reiniciando...
Thread 5 pesquisou o CNPJ 51.017.442/0001-81
Erro no carregamento da Thread 6, reiniciando...
Thread 3 pesquisou o CNPJ 51.017.442/0001-81
Driver da Thread 2 iniciado com sucesso
Erro no carregamento da Thread 4, reiniciando...
Thread 5 clicou em dados diários
Thread 5 iniciando coleta de dados diários
Thread 2 pesquisou o CNPJ 51.017.442/0001-81
Thread 3 clicou em dados diários
Thread 2 clicou em dados diários
Thread 3 iniciando coleta de dados diários
Thread 2 iniciando coleta de dados diários
Thread 5 finalizou coleta de dados diários
[]
Thread 5 reiniciando procura de dados diários
Thread 5 iniciando coleta de dados diários


Exception in thread Thread-21 (get_inside):
Traceback (most recent call last):
  File "c:\USERS\FRODRIGUES\APPDATA\LOCAL\ANACONDA3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\USERS\FRODRIGUES\APPDATA\LOCAL\ANACONDA3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\frodrigues\AppData\Local\Temp\ipykernel_33936\1110155510.py", line 84, in get_inside
  File "c:\Users\frodrigues\Python\Threading_Rent_Catcher\reusable_all_threads_per_dropdown\get_daily_data.py", line 69, in get_daily_data
    get_daily_data(index, driver, cnpj, stored_name, new_item_to_finish)
  File "c:\Users\frodrigues\Python\Threading_Rent_Catcher\reusable_all_threads_per_dropdown\get_daily_data.py", line 31, in get_daily_data
    dropdown_data_pesquisa.select_by_index(item_to_finish)
  File "c:\USERS\FRODRIGUES\APPDATA\LOCAL\ANACONDA3\Lib\site-packages\selenium\webdriver\support\select.py", line 103, in select_by_index
    raise NoSuchEle

Thread 2 finalizou coleta de dados diários
[]
Thread 2 reiniciando procura de dados diários
Thread 2 iniciando coleta de dados diários
Thread 3 finalizou coleta de dados diários
[]
Thread 3 reiniciando procura de dados diários
Thread 3 iniciando coleta de dados diários


In [ ]:
import pandas as pd
testing = pd.DataFrame(dados_diarios_list)
testing['Mês'] = pd.to_datetime(testing['Mês'], format='%m/%Y')
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)

testing_sorted

In [ ]:
testing_sorted = testing.sort_values(by=['Cnpj', 'Mês'], ignore_index=True)
testing_sorted

In [ ]:
testing_sorted.to_excel('teste.xlsx', index=False)

In [ ]:
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)
testing_sorted

In [ ]:
tamanho = 6
to_int = int(tamanho)

In [ ]:
a = list(range(to_int))

In [ ]:
b = a.pop(0)
print(b)

In [ ]:
import psutil

def kill_webdrivers():
    for process in psutil.process_iter(attrs=['pid', 'name']):
        if process.info['name'] and ('chromedriver' in process.info['name'].lower() or 'geckodriver' in process.info['name'].lower()):
            print(f"Matando processo {process.info['name']} (PID: {process.info['pid']})")
            psutil.Process(process.info['pid']).terminate()  # Encerra o processo

kill_webdrivers()
